# Conditional vs Quantifier Analysis with SAE-Lens

Using proven SAE implementation that actually works.

**Setup**: Runtime → Change runtime type → T4 GPU

## 1. Installation

In [ ]:
# Install SAE-Lens and dependencies
!pip install sae-lens transformer-lens circuitsvis
!pip install --upgrade numpy  # Ensure compatibility

print("✓ SAE-Lens installed")

In [ ]:
# Clone your repository for the dataset
!rm -rf SAE_Functional
!git clone https://github.com/ychleee/SAE_Functional.git
%cd SAE_Functional

# Mount Drive for results
from google.colab import drive
drive.mount('/content/drive')

RESULTS_DIR = '/content/drive/MyDrive/sae_lens_results'
!mkdir -p {RESULTS_DIR}
print(f"Results will be saved to: {RESULTS_DIR}")

## 2. Load Dataset

In [ ]:
import csv
import numpy as np
import torch
from collections import Counter

# Load the balanced dataset
texts = []
types = []

dataset_file = 'data/processed/quantifier_conditional_balanced.csv'
with open(dataset_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        texts.append(row['text'])
        types.append(row['type'])

types_array = np.array(types)
type_counts = Counter(types)

# Group counts
conditionals = type_counts.get('conditional', 0)
quantifiers = sum(v for k, v in type_counts.items() if 'universal' in k)
controls = type_counts.get('control', 0)

print("="*60)
print("DATASET LOADED")
print("="*60)
print(f"Total: {len(texts)} sentences")
print(f"  • Conditionals: {conditionals}")
print(f"  • Quantifiers: {quantifiers}")
print(f"  • Controls: {controls}")

## 3. Load Model with TransformerLens

In [ ]:
from transformer_lens import HookedTransformer
import torch

# Load model with TransformerLens (works better with SAE-Lens)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load Pythia-70m
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-70m",
    device=device
)

print(f"\n✓ Loaded model: {model.cfg.model_name}")
print(f"  Hidden size: {model.cfg.d_model}")
print(f"  Layers: {model.cfg.n_layers}")
print(f"  Attention heads: {model.cfg.n_heads}")

## 4. Extract Activations

In [ ]:
from tqdm import tqdm

def extract_activations_transformer_lens(texts, model, layer=4, batch_size=32):
    """Extract activations using TransformerLens."""
    all_activations = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Extracting"):
        batch = texts[i:i+batch_size]
        
        # Get activations at specific layer
        _, cache = model.run_with_cache(
            batch,
            names_filter=lambda name: name == f"blocks.{layer}.hook_resid_post"
        )
        
        # Get residual stream after layer
        activations = cache[f"blocks.{layer}.hook_resid_post"]
        
        # Mean pool over sequence length
        pooled = activations.mean(dim=1)
        all_activations.append(pooled.cpu())
    
    return torch.cat(all_activations, dim=0)

print("Extracting activations...")
activations = extract_activations_transformer_lens(texts, model, layer=4)
print(f"\n✓ Activations shape: {activations.shape}")

## 5. Train SAE with SAE-Lens

In [ ]:
from sae_lens import SAE, SAEConfig

# Configure SAE with proven settings
cfg = SAEConfig(
    # Architecture
    d_in=model.cfg.d_model,  # Input dimension from model
    d_sae=model.cfg.d_model * 16,  # 16x expansion (recommended)
    
    # Sparsity
    k=32,  # Target ~32 active features
    dead_feature_window=5000,  # Track dead features
    dead_feature_threshold=1e-8,
    
    # Training
    l1_coefficient=8e-5,  # Tuned for Pythia
    lp_norm=1,  # L1 penalty
    
    # Advanced features
    use_ghost_grads=True,  # Prevent dead features
    normalize_sae_decoder=True,
    scale_sparsity_penalty_by_decoder_norm=True,
    
    # Device
    device=device,
    dtype=torch.float32
)

print("SAE Configuration:")
print(f"  Input dim: {cfg.d_in}")
print(f"  SAE dim: {cfg.d_sae} ({cfg.d_sae // cfg.d_in}x expansion)")
print(f"  Target sparsity: ~{cfg.k} features")
print(f"  Ghost gradients: {cfg.use_ghost_grads}")
print(f"  L1 coefficient: {cfg.l1_coefficient}")

In [ ]:
# Initialize and train SAE
sae = SAE(cfg)

# Create optimizer
optimizer = torch.optim.Adam(
    sae.parameters(),
    lr=3e-4,
    betas=(0.9, 0.999)
)

# Training with proper sparsity
print("\nTraining SAE with ghost gradients...")
print("="*60)

acts_tensor = activations.to(device)
losses = []

for step in range(5000):  # More steps for better convergence
    # Forward pass
    sae_out = sae(acts_tensor)
    
    # Compute loss (includes ghost gradient handling)
    l_rec = (sae_out.sae_out - acts_tensor).pow(2).mean()
    l_l1 = sae_out.feature_acts.abs().mean()
    loss = l_rec + cfg.l1_coefficient * l_l1
    
    # Ghost gradients for dead features
    if cfg.use_ghost_grads and step > 100:
        # SAE-Lens handles this internally
        dead_features = (sae_out.feature_acts.sum(0) == 0)
        if dead_features.any():
            # Re-initialize dead features
            sae.handle_dead_features(dead_features)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Normalize decoder
    if cfg.normalize_sae_decoder:
        sae.set_decoder_norm_to_unit_norm()
    
    losses.append(loss.item())
    
    if step % 1000 == 0:
        n_active = (sae_out.feature_acts > 0).float().sum(1).mean()
        n_dead = (sae_out.feature_acts.sum(0) == 0).sum()
        print(f"Step {step}: Loss={loss:.4f} (Rec:{l_rec:.4f}, L1:{l_l1:.4f})")
        print(f"         Active features: {n_active:.1f}, Dead: {n_dead}/{cfg.d_sae}")

print("="*60)
print("✓ Training complete!")

## 6. Analyze Feature Overlap

In [ ]:
# Encode all activations with trained SAE
sae.eval()
with torch.no_grad():
    sae_out = sae(acts_tensor)
    features = sae_out.feature_acts.cpu().numpy()

print(f"Encoded features shape: {features.shape}")
print(f"Sparsity: {(features > 0).mean():.3%} of features active")
print(f"Features per sample: {(features > 0).sum(1).mean():.1f} ± {(features > 0).sum(1).std():.1f}")

# This should show VARIABLE sparsity, not constant!

In [ ]:
# Separate by type
cond_mask = types_array == 'conditional'
quant_mask = np.isin(types_array, [
    'pure_universal', 'restricted_universal',
    'negative_universal', 'generic_universal', 'any_universal'
])
ctrl_mask = types_array == 'control'

cond_features = features[cond_mask]
quant_features = features[quant_mask]
ctrl_features = features[ctrl_mask]

print("Features by type:")
print(f"  Conditionals: {(cond_features > 0).sum(1).mean():.1f} active features")
print(f"  Quantifiers: {(quant_features > 0).sum(1).mean():.1f} active features")
print(f"  Controls: {(ctrl_features > 0).sum(1).mean():.1f} active features")

In [ ]:
# Find differential features
def analyze_feature_overlap(cond_features, quant_features, ctrl_features, threshold=0.1):
    """Analyze which features are characteristic of each type."""
    
    # Calculate activation frequency for each feature
    cond_freq = (cond_features > 0).mean(0)
    quant_freq = (quant_features > 0).mean(0)
    ctrl_freq = (ctrl_features > 0).mean(0)
    
    # Find features more active in conditionals/quantifiers than controls
    cond_specific = (cond_freq - ctrl_freq > threshold) & (cond_freq > quant_freq)
    quant_specific = (quant_freq - ctrl_freq > threshold) & (quant_freq > cond_freq)
    shared = (cond_freq - ctrl_freq > threshold) & (quant_freq - ctrl_freq > threshold)
    
    # Get indices
    cond_indices = np.where(cond_specific)[0]
    quant_indices = np.where(quant_specific)[0]
    shared_indices = np.where(shared)[0]
    
    return {
        'conditional_specific': cond_indices,
        'quantifier_specific': quant_indices,
        'shared': shared_indices,
        'cond_freq': cond_freq,
        'quant_freq': quant_freq,
        'ctrl_freq': ctrl_freq
    }

# Analyze
results = analyze_feature_overlap(cond_features, quant_features, ctrl_features)

print("\n" + "="*60)
print("FEATURE OVERLAP ANALYSIS")
print("="*60)
print(f"\n📊 Characteristic Features:")
print(f"  • Conditional-specific: {len(results['conditional_specific'])}")
print(f"  • Quantifier-specific: {len(results['quantifier_specific'])}")
print(f"  • Shared logical features: {len(results['shared'])}")

total = len(results['conditional_specific']) + len(results['quantifier_specific']) + len(results['shared'])
if total > 0:
    overlap_pct = len(results['shared']) / total * 100
    print(f"\n🎯 Overlap: {overlap_pct:.1f}% of characteristic features are shared")

In [ ]:
# Show top shared features
if len(results['shared']) > 0:
    print("\n🔍 Top Shared Features:")
    print("="*60)
    
    # Sort by average activation frequency
    shared_scores = [
        (idx, (results['cond_freq'][idx] + results['quant_freq'][idx]) / 2)
        for idx in results['shared']
    ]
    shared_scores.sort(key=lambda x: x[1], reverse=True)
    
    for i, (feat_idx, score) in enumerate(shared_scores[:10]):
        print(f"\n{i+1}. Feature {feat_idx} (score: {score:.3f})")
        print(f"   Conditional: {results['cond_freq'][feat_idx]:.2%} of sentences")
        print(f"   Quantifier: {results['quant_freq'][feat_idx]:.2%} of sentences")
        print(f"   Control: {results['ctrl_freq'][feat_idx]:.2%} of sentences")
        
        # Find example sentences
        cond_acts = cond_features[:, feat_idx]
        quant_acts = quant_features[:, feat_idx]
        
        # Get top activating conditional
        cond_texts = [t for t, ty in zip(texts, types_array) if ty == 'conditional']
        if len(cond_texts) > 0 and cond_acts.max() > 0:
            top_cond_idx = cond_acts.argmax()
            print(f"   Example (Cond): {cond_texts[top_cond_idx][:80]}...")
        
        # Get top activating quantifier
        quant_texts = [t for t, ty in zip(texts, types_array) if 'universal' in ty]
        if len(quant_texts) > 0 and quant_acts.max() > 0:
            top_quant_idx = quant_acts.argmax()
            print(f"   Example (Quant): {quant_texts[top_quant_idx][:80]}...")
else:
    print("\n⚠️ No shared features found.")
    print("This suggests conditionals and quantifiers use distinct representations.")

## 7. Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot sparsity distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Sparsity by type
axes[0].hist((cond_features > 0).sum(1), bins=30, alpha=0.5, label='Conditional', color='blue')
axes[0].hist((quant_features > 0).sum(1), bins=30, alpha=0.5, label='Quantifier', color='green')
axes[0].hist((ctrl_features > 0).sum(1), bins=30, alpha=0.5, label='Control', color='gray')
axes[0].set_xlabel('Number of Active Features')
axes[0].set_ylabel('Count')
axes[0].set_title('Sparsity Distribution')
axes[0].legend()

# Feature activation frequencies
axes[1].plot(results['cond_freq'], 'b-', alpha=0.5, label='Conditional')
axes[1].plot(results['quant_freq'], 'g-', alpha=0.5, label='Quantifier')
axes[1].plot(results['ctrl_freq'], 'gray', alpha=0.5, label='Control')
axes[1].set_xlabel('Feature Index')
axes[1].set_ylabel('Activation Frequency')
axes[1].set_title('Feature Activation Patterns')
axes[1].legend()
axes[1].set_ylim([0, 1])

# Overlap visualization
categories = ['Cond-specific', 'Shared', 'Quant-specific']
counts = [
    len(results['conditional_specific']),
    len(results['shared']),
    len(results['quantifier_specific'])
]
colors = ['blue', 'yellow', 'green']
axes[2].bar(categories, counts, color=colors)
axes[2].set_ylabel('Number of Features')
axes[2].set_title('Feature Overlap Summary')

plt.suptitle('SAE-Lens Feature Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/sae_lens_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualization saved to {RESULTS_DIR}/sae_lens_analysis.png")

## 8. Save Results

In [ ]:
import json
from datetime import datetime

# Save model
torch.save(sae.state_dict(), f'{RESULTS_DIR}/sae_lens_model.pt')

# Save results
results_json = {
    'timestamp': datetime.now().isoformat(),
    'dataset': {
        'total': len(texts),
        'conditionals': int(conditionals),
        'quantifiers': int(quantifiers),
        'controls': int(controls)
    },
    'sae_config': {
        'd_in': cfg.d_in,
        'd_sae': cfg.d_sae,
        'k': cfg.k,
        'l1_coefficient': cfg.l1_coefficient,
        'use_ghost_grads': cfg.use_ghost_grads
    },
    'results': {
        'conditional_specific': len(results['conditional_specific']),
        'quantifier_specific': len(results['quantifier_specific']),
        'shared': len(results['shared']),
        'overlap_percentage': overlap_pct if total > 0 else 0,
        'avg_active_features': (features > 0).sum(1).mean().item(),
        'sparsity': (features > 0).mean().item()
    },
    'shared_features': results['shared'].tolist()[:20]  # Top 20 shared features
}

with open(f'{RESULTS_DIR}/results.json', 'w') as f:
    json.dump(results_json, f, indent=2)

print("\n" + "="*60)
print("✅ ANALYSIS COMPLETE")
print("="*60)
print(f"\nResults saved to: {RESULTS_DIR}")
print("\nKey findings:")
print(f"  • Sparsity achieved: {(features > 0).sum(1).mean():.1f} features (variable!)")
print(f"  • Feature overlap: {len(results['shared'])} shared features")
print(f"  • Overlap percentage: {overlap_pct:.1f}%" if total > 0 else "  • No differential features found")